In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
# Load TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="emotion_detection_model.tflite")
interpreter.allocate_tensors()

In [3]:
# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [4]:
# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [5]:
# Function to predict emotion on a face using TensorFlow Lite model
def predict_emotion(face_roi):
    # Resize and normalize the face region
    resized_face = cv2.resize(face_roi, (48, 48)) / 255.0

    # Reshape for model input
    input_data = np.expand_dims(resized_face, axis=0)
    input_data = np.expand_dims(input_data, axis=-1)

    # Ensure input data type is FLOAT32
    input_data = input_data.astype(np.float32)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Get the predicted emotion
    emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    predicted_emotion = emotion_labels[np.argmax(output_data)]

    return predicted_emotion

In [6]:
# Function to process video frames from camera
def process_video():
    # Open the camera
    cap = cv2.VideoCapture(0)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, 1.1, 4)

        # Iterate over detected faces
        for (x, y, w, h) in faces:
            # Extract the face region
            face_roi = gray_frame[y:y+h, x:x+w]

            # Predict emotion on the face using TensorFlow Lite model
            emotion = predict_emotion(face_roi)

            # Draw bounding box around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Display the predicted emotion
            cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Facial Emotion Recognition", frame)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

In [7]:
# Call the function to process video frames
process_video()